In [ ]:
import pandas as pd
df = pd.read_excel('/Users/souadmouajel/Desktop/Ironhack/lab-sessions/week-2/project-week2/GSAF5.xls')
df.head()

In [ ]:
df.info()

In [ ]:
# Challenges of Time feature: 
# 1 . alot of missing values
print(f"sum_of_missing_values: {df["Time"].isnull().sum()}")
print()
missing_percent = df["Time"].isnull().sum() / len(df) * 100
print(f"Percentage of missing values: {missing_percent:.2f}%")
print ()
print(f"Huge format variations: see the last 50 value counts") 
df["Time"].value_counts().tail(50)


In [ ]:
# Two steps clean and transform the Time featuers
# First, define a function to replace strange formates with valid ones using reg method (valid method can be strin like Late Afternoon, keep the valid ones 
# but null values have'nt replaced because they are a lot so we avoided the bias"
import pandas as pd
import re

def clean_time(value):
    if pd.isna(value):
        return None

    value = str(value).strip().lower()

    # Remove useless or unclear values
    if value in ["?", "am", "pm", "unknown", "not stated", "n/a", "na"]:
        return None

    # Clean formats like "after 1200hr", "11.30hr", "15.5", etc.
    match = re.search(r'(\d{1,2})[h:.]?(\d{2})?', value)

    if match:
        hour = match.group(1)
        minute = match.group(2) if match.group(2) else "00"
        return f"{hour.zfill(2)}:{minute.zfill(2)}"

    # Keep known phrases like "Morning", "Afternoon", etc.
    return value.title()
from datetime import datetime

# clean time results:
df['Cleaned_Time'] = df['Time'].apply(clean_time)
print(df['Cleaned_Time'].tail(60))

In [ ]:
# Second is to transform the inputs of so the datetime. 
# New function will be developed then it will applied on Time but the result will be saved on new feature called later Day_Part 
from datetime import datetime

def precise_time_to_day_part(value):
    if value is None:
        return None  # Keep missing as None

    # Known descriptive phrases to keep untouched
    descriptive_parts = [
        "Early Morning", "Morning", "Midday", "Early Afternoon",
        "Late Afternoon", "Afternoon", "Evening", "Dusk",
        "Night", "Late Night"
    ]
    
    if isinstance(value, str) and value.title() in descriptive_parts:
        return value.title()

    try:
        # Try parsing standard time like "14:30"
        time = datetime.strptime(value, "%H:%M").time()
        hour = time.hour
        minute = time.minute

        if 5 <= hour < 8:
            return "Early Morning"
        elif 8 <= hour < 12:
            return "Morning"
        elif hour == 12 and minute == 0:
            return "Midday"
        elif 12 <= hour < 15:
            return "Early Afternoon"
        elif 15 <= hour < 17:
            return "Late Afternoon"
        elif 17 <= hour < 19:
            return "Evening"
        elif 19 <= hour < 20:
            return "Dusk"
        elif 20 <= hour < 24:
            return "Night"
        else:  # 00:00 to before 5:00
            return "Late Night"
    except:
        return None  # Unrecognized values go to None
df['Cleaned_Time'] = df['Time'].apply(clean_time)
df['Day_Part'] = df['Cleaned_Time'].apply(precise_time_to_day_part)
print(df['Day_Part'].value_counts())

In [ ]:
df["Date"].isnull().sum()

In [ ]:
# Challenges of Date feature: 
# 1 . There are no of missing values but we have to transform the values
print(f"sum_of_missing_values: {df["Date"].isnull().sum()}")
print()
missing_percent = df["Date"].isnull().sum() / len(df) * 100
print(f"Percentage of missing values: {missing_percent:.2f}%")
print ()
print(f"Huge format variations: see the last 50 value counts") 
df["Date"].value_counts().tail(50)

In [ ]:
# In order to transform the Date column first we have to make sure "Date" column is in datetime format,
# So Text like "Unknown"; Dates missing month or day (e.g., "2023" or "March 2023"), Any other invalid format
df['Cleaned_Date'] = pd.to_datetime(df['Date'], errors='coerce')
# Second we have to define the lables, we choose range date lables rather than season ones becuase the seasons can be different accoring to the country
def get_date_range_label(date):
    if pd.isna(date):
        return "No Date"
    
    day_of_year = date.timetuple().tm_yday  # Day number within the year

    # Margins based on the year of the date
    spring_start = datetime(date.year, 3, 21).timetuple().tm_yday
    summer_start = datetime(date.year, 6, 21).timetuple().tm_yday
    autumn_start = datetime(date.year, 9, 21).timetuple().tm_yday
    winter_start = datetime(date.year, 12, 21).timetuple().tm_yday

    if spring_start <= day_of_year < summer_start:
        return "21-03 to 21-06"
    elif summer_start <= day_of_year < autumn_start:
        return "21-06 to 21-09"
    elif autumn_start <= day_of_year < winter_start:
        return "21-09 to 21-12"
    else:
        return "21-12 to 21-03"

# Apply
df['Date_Range_Label'] = df['Cleaned_Date'].apply(get_date_range_label)

print(df[['Date', 'Cleaned_Date', 'Date_Range_Label']].head(5))
print ()
date_range_missing_values = df['Date_Range_Label'].value_counts().head(10)
print (f"Note: that we have now in Date_Range_Label some missing values {date_range_missing_values} called No Date")

In [ ]:
# Challenges of Injury feature: 
# 1 . There are few of missing values but variance is very high
# Challenges of Injury feature:
print(f"Sum of missing values: {df['Injury'].isnull().sum()}\n")

missing_percent = df["Injury"].isnull().sum() / len(df) * 100
print(f"Percentage of missing values: {missing_percent:.2f}%\n")

print("Huge format variations: see the last 50 value counts")
counts = df["Injury"].value_counts()
print(counts.tail(50))

print("\n")

# Filter where count is exactly 1
single_occurrences = counts[counts == 1]

# Show how many have count = 1
print(f"Number of unique injuries that appear only once: {len(single_occurrences)}")


In [ ]:
# In order to clean column Injury we went in two steps first:
# Purpose:
# Clean the "Injury" column text so that entries with the same injury but different cases, extra spaces, or special characters are standardized.
# for example: Lowercase: "Sprain" and "sprain" should be treated the same.
# Also Strip: Removes accidental spaces around text (" sprain ", "sprain ").
# Replace: Remove anything that is not a lowercase letter or space — this deletes numbers, punctuation, symbols, etc.
# For example, "Sprain#1" → "sprain"
# So we difned new column called Injury_clean using reex technic
df["Injury_clean"] = (
    df["Injury"]
    .str.lower()
    .str.strip()
    .str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-letter characters
)

# Quick grouping preview
print(df["Injury_clean"].value_counts())

In [ ]:
# Second step we define new function called simplify_injury to recategorize all the the values in 7 categories,
# and generated new column called Injury grouped
def simplify_injury(text):
    if pd.isna(text):
        return "unknown"
    text = text.lower()
    if "fatal" in text:
        return "fatal"
    elif "foot" in text:
        return "foot injury"
    elif "leg" in text:
        return "leg injury"
    elif "hand" in text:
        return "hand injury"
    elif "no injury" in text:
        return "no injury"
    else:
        return "other"

df["Injury_grouped"] = df["Injury"].apply(simplify_injury)
print(df["Injury_grouped"].value_counts())